# Generate Bollinger bands using daily dedicated service - F#

### Overview
Bollinger bands were created by John Bollinger, they are envelopes and plotted around the moving average. The width of the bands is based on the standard deviaton of the closing prices from a moving average. 

The purpose of Bollinger Bands is to provide a relative definition of high and low prices of a market. By definition, prices are high at the upper band and low at the lower band.

This sample demonstrates how to request and plot **on-demand** Bollinger bands from a daily data service.

### Inputs/outputs
Bollinger bands indicator requires instrument's identifier,date time intervals and number of periods as per inputs and returns data points for the following components:
1. *Middle band* = n-period moving average
2. *Upper band* = Middle band + (k * n-period standard deviation)
3. *Lower band* = Middle band - (k * n-period standard deviation)

Where:
* *n: number of periods*
* *k: factor to apply to the standard deviation value, k=2 as default value*

This sample shows how to request and plot a simple Bollinger bands graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve Bollinger bands daily data from the hosted service. The queried endpoint in this script are:
* *BollingerBandsService*: to directly retrieve bollinger bands daily data reponse from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily Bollinger bands sample
### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
open Systemathics.Apis.Type.Shared.V1
open Systemathics.Apis.Services.DailyAnalytics.V1
open Google.Protobuf.WellKnownTypes
open Google.Type
open Grpc.Core
open Grpc.Net.Client
open XPlot.Plotly
open XPlot.Plotly.Interactive

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
let token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN")
let api = System.Environment.GetEnvironmentVariable("GRPC_APIS")

// Add token to the request header
let headers = new Metadata()
headers.Add("Authorization", $"Bearer {token}")

// Define API entry
let channel = GrpcChannel.ForAddress($"https://{api}")

In [ ]:
// Display authentication token 
token
|> display

### Step 3: Retrieve data
To request *daily Bollinger bands* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The Bollinger bands parameters

#### 3.1 Instrument selection

In [ ]:
// Set instrument
let ticker = "AAPL"
let exchange = "XNGS"

#### 3.2 Bollinger bands parameters

In [ ]:
// Set parameters
let length = 100
let deviation = 0.4

#### 3.3 Time period selection

In [ ]:
// Set data interval (we are using Google date time format)
let dateIntervals = 
  new DateInterval(
    StartDate = new Date ( Year = 2019, Month = 01, Day = 01 ),
    EndDate = new Date ( Year = 2020, Month = 12, Day = 31 )
  )

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
let constraints = new Constraints() 
constraints.DateIntervals.Add(dateIntervals)

#### 3.4 Request creation
The following code snippets creates *gRPC client*, process daily Bollinger bands request and returns the request reply:

In [ ]:
// Instantiate the daily Bollinger bands service
let service = new DailyBollingerService.DailyBollingerServiceClient(channel)

// Create and process the daily Bollinger bands  request
let request = 
    new DailyBollingerRequest(
        Identifier = new Identifier ( Exchange = exchange, Ticker = ticker ),
        Length = length, 
        Deviation = deviation,
        Constraints = constraints
    )

let reply = service.DailyBollinger(request, headers)

// Visualize requests' results
//reply.Data
//|> display

### Step 4: Visualize data

#### 4.1 Select data

In [ ]:
// Extract the price and the Bollinger bands
let prices  = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value)))
let uppers  = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Upper)))
let middles = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Middle)))
let lowers  = (reply.Data |> Seq.map (fun t -> (new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Lower)))

#### 4.2 Plot Bollinger bands
Display Bollinger bands over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.

In [ ]:
// Plot Bollinger bands
let bollingerGraph = 
    [
        new Scattergl(
            name = "Upper band",
            mode = "lines",
            x = (uppers |>  Seq.map fst),
            y = (uppers |>  Seq.map snd)
        )
        new Scattergl(
            name = "Middle band",
            mode = "lines",
            x = (middles |>  Seq.map fst),
            y = (middles |>  Seq.map snd)
        )
        new Scattergl(
            name = "Lower band",
            mode = "lines",
            x = (lowers |>  Seq.map fst),
            y = (lowers |>  Seq.map snd)
        )
        new Scattergl(
            name = "Price",
            mode = "lines",
            x = (prices |>  Seq.map fst),
            y = (prices |>  Seq.map snd),
            yaxis = "y2"
        )
    ]

let layout = 
    new Layout.Layout(
        title = $"Bollinger bands and daily prices for {ticker}-{exchange}",
        xaxis = new Xaxis( title = "Date"),
        yaxis = new Yaxis (
            title = "Price", 
            autorange = true,
            side = "left"
        ),
        yaxis2 = new Yaxis (
            title = "Bollinger bands", 
            autorange = true,
            side = "right"
        )
    )

Chart.Plot(bollingerGraph,layout)
|> Chart.WithWidth 1500
|> Chart.WithHeight 800
|> display